# About this notebook  
- PyTorch RNN starter code with W&B  
- Pytorch W&B Usage Examples from https://docs.wandb.ai/guides/integrations/pytorch  

If this notebook is helpful, feel free to upvote :)

![](https://raw.githubusercontent.com/google/deluca-lung/main/assets/2020-10-02%20Ventilator%20diagram.svg)

In [1]:
# ====================================================
# Directory settings
# ====================================================
import os

OUTPUT_DIR = './'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [2]:
# ====================================================
# CFG
# ====================================================
class CFG:
    competition='ventilator'
    _wandb_kernel='nakama'
    apex=False
    print_freq=100
    num_workers=4
    model_name='rnn'
    scheduler='CosineAnnealingLR' # ['linear', 'cosine', 'ReduceLROnPlateau', 'CosineAnnealingLR', 'CosineAnnealingWarmRestarts']
    batch_scheduler=False
    #num_warmup_steps=100 # ['linear', 'cosine']
    #num_cycles=0.5 # 'cosine'
    #factor=0.2 # ReduceLROnPlateau
    #patience=4 # ReduceLROnPlateau
    #eps=1e-6 # ReduceLROnPlateau
    T_max=50 # CosineAnnealingLR
    #T_0=50 # CosineAnnealingWarmRestarts
    epochs=5
    max_grad_norm=1000
    gradient_accumulation_steps=1
    hidden_size=64
    lr=5e-3
    min_lr=1e-6
    weight_decay=1e-6
    batch_size=64
    n_fold=5
    trn_fold=[0, 1, 2, 3, 4]
    cate_seq_cols=['R', 'C']
    cont_seq_cols=['time_step', 'u_in', 'u_out'] + ['breath_time', 'u_in_time']
    train=True
    inference=True

In [3]:
# ====================================================
# Library
# ====================================================
import os
import gc
import sys
import json
import time
import math
import random
from datetime import datetime
from collections import Counter, defaultdict

import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

from tqdm.auto import tqdm
import category_encoders as ce

from sklearn import preprocessing
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau

from transformers import AdamW
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup

import warnings
warnings.filterwarnings("ignore")

if CFG.apex:
    from apex import amp

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

2021-10-01 15:51:35.355596: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [4]:
# ====================================================
# wandb
# ====================================================
import wandb

try:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    secret_value_0 = user_secrets.get_secret("wandb_api")
    wandb.login(key=secret_value_0)
    anony = None
except:
    anony = "must"
    print('If you want to use your W&B account, go to Add-ons -> Secrets and provide your W&B access token. Use the Label name as wandb_api. \nGet your W&B access token from here: https://wandb.ai/authorize')

    
def class2dict(f):
    return dict((name, getattr(f, name)) for name in dir(f) if not name.startswith('__'))

run = wandb.init(project="Ventilator-Pressure-Public", 
                 name=CFG.model_name,
                 config=class2dict(CFG),
                 group=CFG.model_name,
                 job_type="train",
                 anonymous=anony)

If you want to use your W&B account, go to Add-ons -> Secrets and provide your W&B access token. Use the Label name as wandb_api. 
Get your W&B access token from here: https://wandb.ai/authorize


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: wandb version 0.12.3 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
2021-10-01 15:51:42.181832: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [5]:
# ====================================================
# Utils
# ====================================================
def get_score(y_trues, y_preds):
    score = mean_absolute_error(y_trues, y_preds)
    return score


def init_logger(log_file=OUTPUT_DIR+'train.log'):
    from logging import getLogger, INFO, FileHandler,  Formatter,  StreamHandler
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=log_file)
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = init_logger()


def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything()

In [6]:
# ====================================================
# Data Loading
# ====================================================
train = pd.read_csv('../input/ventilator-pressure-prediction/train.csv')
test = pd.read_csv('../input/ventilator-pressure-prediction/test.csv')
sub = pd.read_csv('../input/ventilator-pressure-prediction/sample_submission.csv')

for c in ['u_in']:
    train[c] = np.log1p(train[c])
    test[c] = np.log1p(test[c])
    
r_map = {5: 0, 20: 1, 50: 2}
c_map = {10: 0, 20: 1, 50: 2}
train['R'] = train['R'].map(r_map)
test['R'] = test['R'].map(r_map)
train['C'] = train['C'].map(c_map)
test['C'] = test['C'].map(c_map)

display(train.head())
display(test.head())
display(sub.head())


CondaEnvException: Unable to determine environment

Please re-run this command with one of the following options:

* Provide an environment name via --name or -n
* Re-run this command inside an activated conda environment.



,id,breath_id,R,C,time_step,u_in,u_out,pressure
0,1,1,1,2,0.000000,0.080043,0,5.837492
1,2,1,1,2,0.033652,2.964399,0,5.907794
2,3,1,1,2,0.067514,3.157395,0,7.876254
3,4,1,1,2,0.101542,3.170056,0,11.742872
4,5,1,1,2,0.135756,3.271690,0,12.234987


,id,breath_id,R,C,time_step,u_in,u_out
0,1,0,0,1,0.000000,0.000000,0
1,2,0,0,1,0.031904,2.141835,0
2,3,0,0,1,0.063827,2.750578,0
3,4,0,0,1,0.095751,3.101470,0
4,5,0,0,1,0.127644,3.307654,0


,id,pressure
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0


In [7]:
# ====================================================
# FE
# ====================================================
def add_feature(df):
    # breath_time
    df['breath_time'] = df['time_step'] - df['time_step'].shift(1)
    df.loc[df['time_step'] == 0, 'breath_time'] = 0
    # u_in_time
    df['u_in_time'] = df['u_in'] - df['u_in'].shift(1)
    df.loc[df['time_step'] == 0, 'u_in_time'] = 0
    return df


train = add_feature(train)
test = add_feature(test)

In [8]:
# ====================================================
# CV split
# ====================================================
Fold = GroupKFold(n_splits=5)
groups = train['breath_id'].values
for n, (train_index, val_index) in enumerate(Fold.split(train, train['pressure'], groups)):
    train.loc[val_index, 'fold'] = int(n)
train['fold'] = train['fold'].astype(int)
print(train.groupby('fold').size())

fold
0    1207200
1    1207200
2    1207200
3    1207200
4    1207200
dtype: int64


In [9]:
# ====================================================
# Dataset
# ====================================================
class TrainDataset(Dataset):
    def __init__(self, df):
        self.df = df
        self.groups = df.groupby('breath_id').groups
        self.keys = list(self.groups.keys())
        
    def __len__(self):
        return len(self.groups)

    def __getitem__(self, idx):
        indexes = self.groups[self.keys[idx]]
        df = self.df.iloc[indexes]
        cate_seq_x = torch.LongTensor(df[CFG.cate_seq_cols].values)
        cont_seq_x = torch.FloatTensor(df[CFG.cont_seq_cols].values)
        u_out = torch.LongTensor(df['u_out'].values)
        label = torch.FloatTensor(df['pressure'].values)
        return cate_seq_x, cont_seq_x, u_out, label
    

class TestDataset(Dataset):
    def __init__(self, df):
        self.df = df
        self.groups = df.groupby('breath_id').groups
        self.keys = list(self.groups.keys())
        
    def __len__(self):
        return len(self.groups)

    def __getitem__(self, idx):
        indexes = self.groups[self.keys[idx]]
        df = self.df.iloc[indexes]
        cate_seq_x = torch.LongTensor(df[CFG.cate_seq_cols].values)
        cont_seq_x = torch.FloatTensor(df[CFG.cont_seq_cols].values)
        return cate_seq_x, cont_seq_x

In [10]:
# ====================================================
# Model
# ====================================================
class CustomModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg
        self.hidden_size = cfg.hidden_size
        self.r_emb = nn.Embedding(3, 2, padding_idx=0)
        self.c_emb = nn.Embedding(3, 2, padding_idx=0)
        self.seq_emb = nn.Sequential(
            nn.Linear(4 + len(cfg.cont_seq_cols), self.hidden_size),
            nn.LayerNorm(self.hidden_size),
            nn.ReLU(),
            nn.Dropout(0.2),
        )
        self.lstm = nn.LSTM(self.hidden_size, self.hidden_size, 
                            dropout=0.2, batch_first=True, bidirectional=True)
        self.head = nn.Sequential(
            nn.Linear(self.hidden_size * 2, self.hidden_size * 2),
            nn.LayerNorm(self.hidden_size * 2),
            nn.ReLU(),
            nn.Dropout(0.),
            nn.Linear(self.hidden_size * 2, 1),
        )
        for n, m in self.named_modules():
            if isinstance(m, nn.LSTM):
                print(f'init {m}')
                for param in m.parameters():
                    if len(param.shape) >= 2:
                        nn.init.orthogonal_(param.data)
                    else:
                        nn.init.normal_(param.data)
            elif isinstance(m, nn.GRU):
                print(f"init {m}")
                for param in m.parameters():
                    if len(param.shape) >= 2:
                        init.orthogonal_(param.data)
                    else:
                        init.normal_(param.data)

    def forward(self, cate_seq_x, cont_seq_x):
        bs = cont_seq_x.size(0)
        r_emb = self.r_emb(cate_seq_x[:,:,0]).view(bs, 80, -1)
        c_emb = self.c_emb(cate_seq_x[:,:,1]).view(bs, 80, -1)
        seq_x = torch.cat((r_emb, c_emb, cont_seq_x), 2)
        seq_emb = self.seq_emb(seq_x)
        seq_emb, _ = self.lstm(seq_emb)
        output = self.head(seq_emb).view(bs, -1)
        return output

In [11]:
# ====================================================
# helper function
# ====================================================
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (remain %s)' % (asMinutes(s), asMinutes(rs))


def train_fn(fold, train_loader, model, criterion, optimizer, epoch, scheduler, device):
    model.train()
    losses = AverageMeter()
    start = end = time.time()
    for step, (cate_seq_x, cont_seq_x, u_out, y) in enumerate(train_loader):
        loss_mask = u_out == 0
        cate_seq_x, cont_seq_x, y = cate_seq_x.to(device), cont_seq_x.to(device), y.to(device)
        batch_size = cont_seq_x.size(0)
        pred = model(cate_seq_x, cont_seq_x)
        loss = 2. * criterion(pred[loss_mask], y[loss_mask]) + criterion(pred[loss_mask == 0], y[loss_mask == 0])
        losses.update(loss.item(), batch_size)
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        if CFG.apex:
            with amp.scale_loss(loss, optimizer) as scaled_loss:
                scaled_loss.backward()
        else:
            loss.backward()
        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), CFG.max_grad_norm)
        if (step + 1) % CFG.gradient_accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()
            if CFG.batch_scheduler:
                scheduler.step()
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(train_loader)-1):
            print('Epoch: [{0}][{1}/{2}] '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  'Grad: {grad_norm:.4f}  '
                  'LR: {lr:.6f}  '
                  .format(
                   epoch+1, step, len(train_loader),
                   remain=timeSince(start, float(step+1)/len(train_loader)),
                   loss=losses,
                   grad_norm=grad_norm,
                   lr=scheduler.get_lr()[0],
                   ))
        wandb.log({f"[fold{fold}] loss": losses.val,
                   f"[fold{fold}] lr": scheduler.get_lr()[0]})
    return losses.avg


def valid_fn(valid_loader, model, criterion, device):
    model.eval()
    preds = []
    losses = AverageMeter()
    start = end = time.time()
    for step, (cate_seq_x, cont_seq_x, u_out, y) in enumerate(valid_loader):
        loss_mask = u_out == 0
        cate_seq_x, cont_seq_x, y = cate_seq_x.to(device), cont_seq_x.to(device), y.to(device)
        batch_size = cont_seq_x.size(0)
        with torch.no_grad():
            pred = model(cate_seq_x, cont_seq_x)
        loss = 2. * criterion(pred[loss_mask], y[loss_mask]) + criterion(pred[loss_mask == 0], y[loss_mask == 0])
        losses.update(loss.item(), batch_size)
        preds.append(pred.view(-1).detach().cpu().numpy())
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(valid_loader)-1):
            print('EVAL: [{0}/{1}] '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  .format(
                   step, len(valid_loader),
                   remain=timeSince(start, float(step+1)/len(valid_loader)),
                   loss=losses,
                   ))
    preds = np.concatenate(preds)
    return losses.avg, preds


def inference_fn(test_loader, model, device):
    model.eval()
    model.to(device)
    preds = []
    tk0 = tqdm(enumerate(test_loader), total=len(test_loader))
    for step, (cate_seq_x, cont_seq_x) in tk0:
        cate_seq_x, cont_seq_x = cate_seq_x.to(device), cont_seq_x.to(device)
        with torch.no_grad():
            pred = model(cate_seq_x, cont_seq_x)
        preds.append(pred.view(-1).detach().cpu().numpy())
    preds = np.concatenate(preds)
    return preds

In [12]:
# ====================================================
# train loop
# ====================================================
def train_loop(folds, fold):

    LOGGER.info(f"========== fold: {fold} training ==========")

    # ====================================================
    # loader
    # ====================================================
    trn_idx = folds[folds['fold'] != fold].index
    val_idx = folds[folds['fold'] == fold].index
    
    train_folds = train.loc[trn_idx].reset_index(drop=True)
    valid_folds = train.loc[val_idx].reset_index(drop=True)
    y_true = valid_folds['pressure'].values
    non_expiratory_phase_val_idx = valid_folds[valid_folds['u_out'] == 0].index # The expiratory phase is not scored

    train_dataset = TrainDataset(train_folds)
    valid_dataset = TrainDataset(valid_folds)

    train_loader = DataLoader(train_dataset,
                              batch_size=CFG.batch_size,
                              shuffle=True,
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=True)
    valid_loader = DataLoader(valid_dataset,
                              batch_size=CFG.batch_size,
                              shuffle=False,
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=False)

    # ====================================================
    # model & optimizer
    # ====================================================
    model = CustomModel(CFG)
    model.to(device)

    optimizer = AdamW(model.parameters(), lr=CFG.lr, weight_decay=CFG.weight_decay)
    num_train_steps = int(len(train_folds) / CFG.batch_size * CFG.epochs)
    
    def get_scheduler(optimizer):
        if CFG.scheduler=='linear':
            scheduler = get_linear_schedule_with_warmup(
                optimizer, num_warmup_steps=CFG.num_warmup_steps, num_training_steps=num_train_steps
            )
        elif CFG.scheduler=='cosine':
            scheduler = get_cosine_schedule_with_warmup(
                optimizer, num_warmup_steps=CFG.num_warmup_steps, num_training_steps=num_train_steps, num_cycles=CFG.num_cycles
            )
        elif CFG.scheduler=='ReduceLROnPlateau':
            scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=CFG.factor, patience=CFG.patience, verbose=True, eps=CFG.eps)
        elif CFG.scheduler=='CosineAnnealingLR':
            scheduler = CosineAnnealingLR(optimizer, T_max=CFG.T_max, eta_min=CFG.min_lr, last_epoch=-1)
        elif CFG.scheduler=='CosineAnnealingWarmRestarts':
            scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=CFG.T_0, T_mult=1, eta_min=CFG.min_lr, last_epoch=-1)
        return scheduler

    scheduler = get_scheduler(optimizer)

    # ====================================================
    # apex
    # ====================================================
    if CFG.apex:
        model, optimizer = amp.initialize(model, optimizer, opt_level='O1', verbosity=0)

    # ====================================================
    # loop
    # ====================================================
    criterion = nn.L1Loss()

    best_score = np.inf

    for epoch in range(CFG.epochs):

        start_time = time.time()

        # train
        avg_loss = train_fn(fold, train_loader, model, criterion, optimizer, epoch, scheduler, device)

        # eval
        avg_val_loss, preds = valid_fn(valid_loader, model, criterion, device)
        
        if isinstance(scheduler, ReduceLROnPlateau):
            scheduler.step(avg_val_loss)
        elif isinstance(scheduler, CosineAnnealingLR):
            scheduler.step()
        elif isinstance(scheduler, CosineAnnealingWarmRestarts):
            scheduler.step()

        # scoring
        score = get_score(y_true[non_expiratory_phase_val_idx], preds[non_expiratory_phase_val_idx])

        elapsed = time.time() - start_time

        LOGGER.info(f'Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  time: {elapsed:.0f}s')
        LOGGER.info(f'Epoch {epoch+1} - MAE Score (without expiratory phase): {score:.4f}')
        wandb.log({f"[fold{fold}] epoch": epoch+1, 
                   f"[fold{fold}] avg_train_loss": avg_loss, 
                   f"[fold{fold}] avg_val_loss": avg_val_loss,
                   f"[fold{fold}] score": score})
        
        if score < best_score:
            best_score = score
            LOGGER.info(f'Epoch {epoch+1} - Save Best Score: {score:.4f} Model')
            torch.save({'model': model.state_dict(),
                        'preds': preds},
                        OUTPUT_DIR+f"fold{fold}_best.pth")
            
    preds = torch.load(OUTPUT_DIR+f"fold{fold}_best.pth", map_location=torch.device('cpu'))['preds']
    valid_folds['preds'] = preds

    torch.cuda.empty_cache()
    gc.collect()
    
    return valid_folds

In [13]:
# ====================================================
# main
# ====================================================
def main():
    
    """
    Prepare: 1.train 2.test
    """
    
    def get_result(result_df):
        preds = result_df['preds'].values
        labels = result_df['pressure'].values
        non_expiratory_phase_val_idx = result_df[result_df['u_out'] == 0].index # The expiratory phase is not scored
        score = get_score(labels[non_expiratory_phase_val_idx], preds[non_expiratory_phase_val_idx])
        LOGGER.info(f'Score (without expiratory phase): {score:<.4f}')
    
    if CFG.train:
        # train 
        oof_df = pd.DataFrame()
        for fold in range(CFG.n_fold):
            if fold in CFG.trn_fold:
                _oof_df = train_loop(train, fold)
                oof_df = pd.concat([oof_df, _oof_df])
                LOGGER.info(f"========== fold: {fold} result ==========")
                get_result(_oof_df)
        # CV result
        LOGGER.info(f"========== CV ==========")
        get_result(oof_df)
        # save result
        oof_df.to_csv(OUTPUT_DIR+'oof_df.csv', index=False)
    
    if CFG.inference:
        test_dataset = TestDataset(test)
        test_loader = DataLoader(test_dataset, batch_size=CFG.batch_size * 2, shuffle=False, num_workers=CFG.num_workers, pin_memory=True)
        for fold in CFG.trn_fold:
            model = CustomModel(CFG)
            path = OUTPUT_DIR+f"fold{fold}_best.pth"
            state = torch.load(path, map_location=torch.device('cpu'))
            model.load_state_dict(state['model'])
            predictions = inference_fn(test_loader, model, device)
            test[f'fold{fold}'] = predictions
            del state, predictions; gc.collect()
            torch.cuda.empty_cache()
        # submission
        test['pressure'] = test[[f'fold{fold}' for fold in range(CFG.n_fold)]].mean(1)
        test[['id', 'pressure']+[f'fold{fold}' for fold in range(CFG.n_fold)]].to_csv(OUTPUT_DIR+'raw_submission.csv', index=False)
        test[['id', 'pressure']].to_csv(OUTPUT_DIR+'submission.csv', index=False)
    
    wandb.finish()

In [14]:
if __name__ == '__main__':
    main()

========== fold: 0 training ==========


init LSTM(64, 64, batch_first=True, dropout=0.2, bidirectional=True)
Epoch: [1][0/943] Elapsed 0m 0s (remain 11m 57s) Loss: 42.1867(42.1867) Grad: 30.7141  LR: 0.005000  
Epoch: [1][100/943] Elapsed 0m 10s (remain 1m 29s) Loss: 9.7079(18.5723) Grad: 27.6273  LR: 0.005000  
Epoch: [1][200/943] Elapsed 0m 21s (remain 1m 18s) Loss: 4.5978(12.1808) Grad: 46.6698  LR: 0.005000  
Epoch: [1][300/943] Elapsed 0m 31s (remain 1m 7s) Loss: 3.6194(9.5902) Grad: 30.7479  LR: 0.005000  
Epoch: [1][400/943] Elapsed 0m 42s (remain 0m 56s) Loss: 3.5849(8.2343) Grad: 15.6006  LR: 0.005000  
Epoch: [1][500/943] Elapsed 0m 52s (remain 0m 46s) Loss: 4.8776(7.3472) Grad: 49.1285  LR: 0.005000  
Epoch: [1][600/943] Elapsed 1m 2s (remain 0m 35s) Loss: 3.1692(6.7095) Grad: 16.0577  LR: 0.005000  
Epoch: [1][700/943] Elapsed 1m 13s (remain 0m 25s) Loss: 3.2046(6.2384) Grad: 18.0597  LR: 0.005000  
Epoch: [1][800/943] Elapsed 1m 23s (remain 0m 14s) Loss: 3.6701(5.8619) Grad: 24.2422  LR: 0.005000  
Epoch: [1][90

Epoch 1 - avg_train_loss: 5.4777  avg_val_loss: 2.8339  time: 120s
Epoch 1 - MAE Score (without expiratory phase): 1.2079
Epoch 1 - Save Best Score: 1.2079 Model


EVAL: [235/236] Elapsed 0m 22s (remain 0m 0s) Loss: 2.4933(2.8339) 
Epoch: [2][0/943] Elapsed 0m 0s (remain 8m 33s) Loss: 2.8398(2.8398) Grad: 17.6497  LR: 0.004990  
Epoch: [2][100/943] Elapsed 0m 10s (remain 1m 30s) Loss: 4.1900(3.2174) Grad: 65.8356  LR: 0.004990  
Epoch: [2][200/943] Elapsed 0m 21s (remain 1m 20s) Loss: 2.9325(3.2332) Grad: 17.1712  LR: 0.004990  
Epoch: [2][300/943] Elapsed 0m 32s (remain 1m 9s) Loss: 3.4572(3.1972) Grad: 26.8499  LR: 0.004990  
Epoch: [2][400/943] Elapsed 0m 42s (remain 0m 57s) Loss: 2.8838(3.1867) Grad: 40.7367  LR: 0.004990  
Epoch: [2][500/943] Elapsed 0m 53s (remain 0m 47s) Loss: 2.8084(3.1564) Grad: 18.4096  LR: 0.004990  
Epoch: [2][600/943] Elapsed 1m 4s (remain 0m 36s) Loss: 3.4701(3.1189) Grad: 47.9236  LR: 0.004990  
Epoch: [2][700/943] Elapsed 1m 14s (remain 0m 25s) Loss: 2.7224(3.0776) Grad: 28.0150  LR: 0.004990  
Epoch: [2][800/943] Elapsed 1m 25s (remain 0m 15s) Loss: 3.5705(3.0570) Grad: 35.5448  LR: 0.004990  
Epoch: [2][900/943]

Epoch 2 - avg_train_loss: 3.0290  avg_val_loss: 2.5013  time: 123s
Epoch 2 - MAE Score (without expiratory phase): 1.0566
Epoch 2 - Save Best Score: 1.0566 Model


...

  0%|          | 0/393 [00:00<?, ?it/s]

init LSTM(64, 64, batch_first=True, dropout=0.2, bidirectional=True)


  0%|          | 0/393 [00:00<?, ?it/s]

[fold0] loss,2.72262
[fold0] lr,0.00489
_runtime,3172
_timestamp,1633106672
_step,23599
[fold0] epoch,5
[fold0] avg_train_loss,2.46165
[fold0] avg_val_loss,2.4972
[fold0] score,1.07109
[fold1] loss,2.25319
[fold1] lr,0.00489


[fold0] loss,█▃▂▂▂▂▂▂▂▂▁▁▁▂▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁
[fold0] lr,████████▇▇▇▇▇▇▇▇▆▆▆▆▆▆▆▆▄▄▄▄▄▄▄▄▁▁▁▁▁▁▁▁
_runtime,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
_timestamp,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
[fold0] epoch,▁▃▅▆█
[fold0] avg_train_loss,█▂▂▁▁
[fold0] avg_val_loss,█▁▂▁▁
[fold0] score,█▁▂▁▂
[fold1] loss,█▄▂▂▂▂▁▂▂▁▂▁▂▂▁▁▂▂▂▁▁▁▁▁▁▂▂▁▁▁▁▁▁▁▂▁▁▁▁▁
[fold1] lr,████████▇▇▇▇▇▇▇▇▆▆▆▆▆▆▆▆▄▄▄▄▄▄▄▄▁▁▁▁▁▁▁▁
